In [0]:
from pyspark.sql import functions as F

In [0]:
fact_bus_ml = (spark.table("hive_metastore.tfm.fact_bus"))

fact_weather_enriched = (
    (spark.table("hive_metastore.tfm.fact_weather_enriched"))
    .where(F.col('weather_station_id') == 'USW00094728')
)

dim_calendar = (spark.table("hive_metastore.tfm.dim_calendar"))

fact_bus_ml.cache()
fact_weather_enriched.cache()
dim_calendar.cache()

DataFrame[date: date, is_holiday: int, is_pre_holiday: int, is_weekend: int, day_of_week: int, month: int]

In [0]:
display(fact_bus_ml.limit(10))

day,bus_route,total_ridership
2020-01-01,B1,6252
2020-01-01,B100,751
2020-01-01,B103,3934
2020-01-01,B11,4123
2020-01-01,B12,3641
2020-01-01,B13,1563
2020-01-01,B14,1452
2020-01-01,B15,7402
2020-01-01,B16,2148
2020-01-01,B17,2886


In [0]:
display(fact_weather_enriched.limit(10))

weather_station_id,date,awnd,prcp,snow,snwd,tmax,tmin,wt01,wt02,wt03,wt04,wt05,wt06,wt08,trange,awnd_bin,prcp_bin,snow_bin,snwd_bin,tmax_bin,tmin_bin,trange_bin,weather_station_date
USW00094728,2020-01-01,6.1155072,0.0,0.0,0.0,-15.0,-17.166666666666668,0,0,0,0,0,0,0,2.166666666666668,cat1,cat0,cat0,cat0,cat0,cat0,cat0,USW00094728/2020-01-01
USW00094728,2020-01-02,3.8624256,0.0,0.0,0.0,-12.555555555555557,-17.444444444444443,0,0,0,0,0,0,0,4.888888888888886,cat1,cat0,cat0,cat0,cat0,cat0,cat1,USW00094728/2020-01-02
USW00094728,2020-01-03,2.414016,96.52,0.0,0.0,-12.555555555555557,-14.055555555555557,1,0,0,0,0,0,1,1.5,cat0,cat2,cat0,cat0,cat0,cat0,cat0,USW00094728/2020-01-03
USW00094728,2020-01-04,3.218688,175.26,0.0,0.0,-11.88888888888889,-15.0,1,1,0,0,0,0,0,3.1111111111111107,cat0,cat2,cat0,cat0,cat0,cat0,cat0,USW00094728/2020-01-04
USW00094728,2020-01-05,8.2076544,0.0,0.0,0.0,-14.666666666666666,-16.833333333333336,0,0,0,0,0,0,0,2.1666666666666696,cat2,cat0,cat0,cat0,cat0,cat0,cat0,USW00094728/2020-01-05
USW00094728,2020-01-06,4.345228800000001,25.4,5.0,0.0,-13.777777777777779,-17.77777777777778,1,0,0,0,0,0,0,4.0,cat1,cat1,cat4,cat0,cat0,cat0,cat1,USW00094728/2020-01-06
USW00094728,2020-01-07,3.7014912,0.0,0.0,0.0,-13.777777777777779,-16.555555555555557,0,0,0,0,0,0,0,2.7777777777777786,cat1,cat0,cat0,cat0,cat0,cat0,cat0,USW00094728/2020-01-07
USW00094728,2020-01-08,6.9201792,0.0,0.0,0.0,-14.666666666666666,-18.944444444444446,0,0,0,0,0,0,1,4.27777777777778,cat2,cat0,cat0,cat0,cat0,cat0,cat1,USW00094728/2020-01-08
USW00094728,2020-01-09,3.218688,0.0,0.0,0.0,-17.166666666666668,-20.5,0,0,0,0,0,0,0,3.333333333333332,cat0,cat0,cat0,cat0,cat0,cat0,cat0,USW00094728/2020-01-09
USW00094728,2020-01-10,4.5061632,0.0,0.0,0.0,-10.666666666666666,-17.444444444444443,0,0,0,0,0,0,0,6.777777777777777,cat1,cat0,cat0,cat0,cat0,cat0,cat3,USW00094728/2020-01-10


1. JOIN con la tabla weather

In [0]:
fact_bus_ml = fact_bus_ml.withColumnRenamed('day', 'date')

In [0]:
fact_bus_ml = (
    fact_bus_ml
    .join(
        fact_weather_enriched,
        on = 'date',
        how = 'left'
    )
    .select(
        F.col('date'),
        F.col('total_ridership'),
        F.col('awnd_bin'),
        F.col('prcp_bin'),
        F.col('snow_bin'),
        F.col('snwd_bin'),
        F.col('tmax_bin'),
        F.col('tmin_bin'),
        F.col('trange_bin')
    )
)

In [0]:
display(fact_bus_ml.limit(10))

date,total_ridership,awnd_bin,prcp_bin,snow_bin,snwd_bin,tmax_bin,tmin_bin,trange_bin
2020-01-01,6252,cat1,cat0,cat0,cat0,cat0,cat0,cat0
2020-01-01,751,cat1,cat0,cat0,cat0,cat0,cat0,cat0
2020-01-01,3934,cat1,cat0,cat0,cat0,cat0,cat0,cat0
2020-01-01,4123,cat1,cat0,cat0,cat0,cat0,cat0,cat0
2020-01-01,3641,cat1,cat0,cat0,cat0,cat0,cat0,cat0
2020-01-01,1563,cat1,cat0,cat0,cat0,cat0,cat0,cat0
2020-01-01,1452,cat1,cat0,cat0,cat0,cat0,cat0,cat0
2020-01-01,7402,cat1,cat0,cat0,cat0,cat0,cat0,cat0
2020-01-01,2148,cat1,cat0,cat0,cat0,cat0,cat0,cat0
2020-01-01,2886,cat1,cat0,cat0,cat0,cat0,cat0,cat0


2. JOIN CALENDAR

In [0]:
fact = fact_bus_ml.alias('fact')
calendar = dim_calendar.alias('calendar')

fact_bus_ml = (
    fact
    .join(
        calendar,
        fact['date'] == calendar['date'],
        how='left'
    )
    .select(
        F.col('fact.date').alias('date'),
        F.col('fact.total_ridership'),
        F.col('fact.awnd_bin'),
        F.col('fact.prcp_bin'),
        F.col('fact.snow_bin'),
        F.col('fact.snwd_bin'),
        F.col('fact.tmax_bin'),
        F.col('fact.tmin_bin'),
        F.col('fact.trange_bin'),
        F.col('calendar.is_holiday'),
        F.col('calendar.is_pre_holiday'),
        F.col('calendar.is_weekend'),
        F.col('calendar.day_of_week'),
        F.col('calendar.month')
    )
)
  

In [0]:
display(fact_bus_ml.limit(10))

date,total_ridership,awnd_bin,prcp_bin,snow_bin,snwd_bin,tmax_bin,tmin_bin,trange_bin,is_holiday,is_pre_holiday,is_weekend,day_of_week,month
2020-01-01,6252,cat1,cat0,cat0,cat0,cat0,cat0,cat0,1,0,0,4,1
2020-01-01,751,cat1,cat0,cat0,cat0,cat0,cat0,cat0,1,0,0,4,1
2020-01-01,3934,cat1,cat0,cat0,cat0,cat0,cat0,cat0,1,0,0,4,1
2020-01-01,4123,cat1,cat0,cat0,cat0,cat0,cat0,cat0,1,0,0,4,1
2020-01-01,3641,cat1,cat0,cat0,cat0,cat0,cat0,cat0,1,0,0,4,1
2020-01-01,1563,cat1,cat0,cat0,cat0,cat0,cat0,cat0,1,0,0,4,1
2020-01-01,1452,cat1,cat0,cat0,cat0,cat0,cat0,cat0,1,0,0,4,1
2020-01-01,7402,cat1,cat0,cat0,cat0,cat0,cat0,cat0,1,0,0,4,1
2020-01-01,2148,cat1,cat0,cat0,cat0,cat0,cat0,cat0,1,0,0,4,1
2020-01-01,2886,cat1,cat0,cat0,cat0,cat0,cat0,cat0,1,0,0,4,1


4. Guardar tabla

In [0]:
fact_bus_ml.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("tfm.fact_bus_ml")